# Safety-Aware Schedule Synthesis via Period Boosting and Compressing

In [1]:
using Plots
using LaTeXStrings
using ControlSystemsBase
using DelimitedFiles

using ControlSafetyBench
using ControlTimingSafety

push!(LOAD_PATH, "../lib")
using Experiments

[ Info: Precompiling ControlTimingSafety [3a76c758-31f1-42cb-b697-12fe755e3b12]
[ Info: Precompiling Experiments [top-level]


## Explore Systems and Set up Parameters

In [2]:
sysc = benchmarks[:F1]

StateSpace{Continuous, Float64}
A = 
 0.0  6.5
 0.0  0.0
B = 
  0.0
 19.68503937007874
C = 
 1.0  0.0
D = 
 0.0

Continuous-time state-space model

In [12]:
H = 100
d_max = 1000
maxwindow = 6
params = (
    RC = (p = 23, d = 1.4, x0 = 100, n = 10),
    F1 = (p = 20, d = 1.2, x0 = 1,   n = 15),
    DC = (p = 23, d = 3.5, x0 = 100, n = 10),
    CS = (p = 27, d = 9.4, x0 = 100, n = 15),
    CC = (p = 28, d = 5.3, x0 = 10,  n = 15))
K = (
    RC = delay_lqr(benchmarks[:RC], 0.001*params[:RC][:p]),
    F1 = delay_lqr(benchmarks[:F1], 0.001*params[:F1][:p]),
    DC = delay_lqr(benchmarks[:DC], 0.001*params[:DC][:p]),
    CS = delay_lqr(benchmarks[:CS], 0.001*params[:CS][:p]),
    CC = delay_lqr(benchmarks[:CC], 0.001*params[:CC][:p]))
K[:RC]

1×3 Matrix{Float64}:
 0.160079  0.214257  0.0221421

## Calculating and Comparing Nominal Trajectories

We compare the two nominal trajectories:
- `nominal1`: the original trajectory with $p$ ms period
- `nominal2`: a trajectory with $1$ ms period that simulates the original nominal behavior with a much finer interval.

The `nominal2` trajectory is simulated with every $p$-th deadline being a hit, and all other deadlines being misses. To simulate the logical execution time (LET) paradigm, where the sampling of system state and application of control input is $p$ ms apart, the `Hold&SkipNext` policy is used to ensure the $p$ ms delay in control input calculation.

In [4]:
let
    nominal1 = nominal_trajectory_1(sysc, params[:F1][:x0], params[:F1][:p], H)
    nominal2 = nominal_trajectory_2(sysc, params[:F1][:x0], params[:F1][:p], H)
    slice_nominal(nominal2, params[:F1][:p]) ≈ nominal1
end

true

In [5]:
nominal = (
    RC = nominal_trajectory_2(benchmarks[:RC], params[:RC][:x0], params[:RC][:p], H),
    F1 = nominal_trajectory_2(benchmarks[:F1], params[:F1][:x0], params[:F1][:p], H),
    DC = nominal_trajectory_2(benchmarks[:DC], params[:DC][:x0], params[:DC][:p], H),
    CS = nominal_trajectory_2(benchmarks[:CS], params[:CS][:x0], params[:CS][:p], H),
    CC = nominal_trajectory_2(benchmarks[:CC], params[:CC][:x0], params[:CC][:p], H))
nominal[:RC]

2×2301 Matrix{Float64}:
 100.0  99.5015  99.0059  98.5132  …  -0.205483  -0.205453  -0.205435
 100.0  99.95    99.8999  99.8497     15.9149    15.902     15.8891

## Synthesizing Constraints and Compare with Non-Period-Boosted Solution

The reference method without any period boosting

In [13]:
ref(s::Symbol) = synthesize_constraints(c2d(benchmarks[s], params[s][:p]*0.001),
    K[s], x_to_z_kill(benchmarks[s], params[s][:x0]), d_max, maxwindow, 
    params[s][:n], H, fullresults=true, nominal=slice_nominal(nominal[s], params[s][:p]))[2];

In [14]:
ref(:RC)

6×6 Matrix{Float64}:
  2.6974e-6  Inf         Inf         Inf         Inf         Inf
  4.0494      2.6974e-6  Inf         Inf         Inf         Inf
  7.2908      4.0494      2.6974e-6  Inf         Inf         Inf
  9.86749     7.2908      4.0494      2.6974e-6  Inf         Inf
 11.8986      9.86749     7.2908      4.0494      2.6974e-6  Inf
 13.483      11.8986      9.86749     7.2908      4.0494      2.6974e-6

Constraint synthesis with period boosting

In [15]:
pb(s::Symbol, newp::Real) = period_boosting(benchmarks[s], K[s], newp,
    x_to_z_kill(benchmarks[s], params[s][:x0]), d_max, maxwindow, params[s][:n], nominal[s]);

Compare two approaches with original period ($23$ ms). There should not be any differences in this case.

In [16]:
pb(:RC, 23) - reff(:RC)

6×6 Matrix{Float64}:
 0.0  NaN    NaN    NaN    NaN    NaN
 0.0    0.0  NaN    NaN    NaN    NaN
 0.0    0.0    0.0  NaN    NaN    NaN
 0.0    0.0    0.0    0.0  NaN    NaN
 0.0    0.0    0.0    0.0    0.0  NaN
 0.0    0.0    0.0    0.0    0.0    0.0

Compare the original with a new period $p=28$ ms. The values below
are the net changes between the two periods. In this case, all constraints have higher deviations.

In [17]:
pb(:RC, 28) - reff(:RC)

6×6 Matrix{Float64}:
 0.926938  NaN         NaN         NaN         NaN         NaN
 1.46229     0.926938  NaN         NaN         NaN         NaN
 1.71857     1.46229     0.926938  NaN         NaN         NaN
 1.78087     1.71857     1.46229     0.926938  NaN         NaN
 1.69665     1.78087     1.71857     1.46229     0.926938  NaN
 1.51259     1.69665     1.78087     1.71857     1.46229     0.926938

## Experiments

In [11]:
res = Dict()

for sys in [:RC, :F1, :DC, :CS, :CC], newp in [15, 28, 40]
    res[(sys, newp)] = pb(sys, newp)
    @info sys newp res[(sys, newp)]
    writedlm("../data/$sys-$newp.csv", round.(res[(sys, newp)], sigdigits=5), '\t')
end

┌ Info: RC
│   newp = 15
│   res[(sys, newp)] =
│    6×6 Matrix{Float64}:
│     1.41163  Inf       Inf       Inf       Inf       Inf
│     1.41163   1.41163  Inf       Inf       Inf       Inf
│     3.88435   1.41163   1.41163  Inf       Inf       Inf
│     6.07337   3.88435   1.41163   1.41163  Inf       Inf
│     7.99598   6.07337   3.88435   1.41163   1.41163  Inf
└     9.65662   7.99598   6.07337   3.88435   1.41163   1.41163
┌ Info: RC
│   newp = 28
│   res[(sys, newp)] =
│    6×6 Matrix{Float64}:
│      0.926941  Inf        Inf        Inf        Inf        Inf
│      5.51169    0.926941  Inf        Inf        Inf        Inf
│      9.00937    5.51169    0.926941  Inf        Inf        Inf
│     11.6484     9.00937    5.51169    0.926941  Inf        Inf
│     13.5953    11.6484     9.00937    5.51169    0.926941  Inf
└     14.9956    13.5953    11.6484     9.00937    5.51169    0.926941
┌ Info: RC
│   newp = 40
│   res[(sys, newp)] =
│    6×6 Matrix{Float64}:
│      3.04462  Inf    